# Med-I-C · AMR-Guard
### Infection Lifecycle Orchestrator — Kaggle Demo

**Steps**
1. Clone repo & install packages
2. Authenticate with Hugging Face
3. Download models
4. Initialise the knowledge base
5. Launch the Streamlit app

## 1 · Environment

In [ ]:
import subprocess, torch

# GPU check
print(subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'],
                     capture_output=True, text=True).stdout.strip())
print(f"PyTorch {torch.__version__} · CUDA {torch.cuda.is_available()}")

In [ ]:
%%bash
# Clone the repo (skip if already present)
if [ ! -d /kaggle/working/Med-I-C ]; then
    git clone https://github.com/benghita/Med-I-C.git /kaggle/working/Med-I-C
else
    echo "Repo already cloned — pulling latest changes"
    git -C /kaggle/working/Med-I-C pull
fi

In [ ]:
%%capture
# Install packages from pyproject.toml dependencies
!pip install -q \
    "langgraph>=0.0.15" "langchain>=0.3.0" langchain-text-splitters langchain-community \
    "chromadb>=0.4.0" sentence-transformers \
    "transformers>=4.50.0" accelerate bitsandbytes \
    streamlit huggingface_hub \
    "pydantic>=2.0" python-dotenv openpyxl pypdf "pandas>=2.0" jq

## 2 · Hugging Face Authentication

Add your token to **Kaggle → Add-ons → Secrets** as `HF_TOKEN`.

Accept model licences before running:
- https://huggingface.co/google/gemma-2-2b-it

In [ ]:
import os
from huggingface_hub import login

try:
    from kaggle_secrets import UserSecretsClient
    hf_token = UserSecretsClient().get_secret("HF_TOKEN")
    print("Token loaded from Kaggle secrets")
except Exception:
    hf_token = os.getenv("HF_TOKEN", "")
    print("Token loaded from environment" if hf_token else "WARNING: No HF_TOKEN found")

if hf_token:
    login(token=hf_token, add_to_git_credential=False)

## 3 · Download Models

In [ ]:
from huggingface_hub import snapshot_download

# Single model used for all agents in the demo
MODEL_ID = "google/gemma-2-2b-it"

print(f"Downloading {MODEL_ID} …")
snapshot_download(
    repo_id=MODEL_ID,
    ignore_patterns=["*.gguf", "*.ot"],   # skip quantised formats we don't need
)
print("Download complete")

In [ ]:
# Embedding model for RAG (small, fast)
from sentence_transformers import SentenceTransformer
SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
print("Embedding model ready")

## 4 · Configure & Initialise

In [ ]:
# Write .env for the Kaggle environment
env_content = f"""
MEDIC_ENV=kaggle
MEDIC_DEFAULT_BACKEND=local
MEDIC_USE_VERTEX=false
MEDIC_QUANTIZATION=4bit

MEDIC_LOCAL_MEDGEMMA_4B_MODEL={MODEL_ID}
MEDIC_LOCAL_MEDGEMMA_27B_MODEL={MODEL_ID}
MEDIC_LOCAL_TXGEMMA_9B_MODEL={MODEL_ID}
MEDIC_LOCAL_TXGEMMA_2B_MODEL={MODEL_ID}

MEDIC_EMBEDDING_MODEL=sentence-transformers/all-MiniLM-L6-v2
MEDIC_DATA_DIR=/kaggle/working/Med-I-C/data
MEDIC_CHROMA_DB_DIR=/kaggle/working/Med-I-C/data/chroma_db
""".strip()

with open("/kaggle/working/Med-I-C/.env", "w") as f:
    f.write(env_content)

print(".env written")

In [ ]:
import sys
sys.path.insert(0, "/kaggle/working/Med-I-C")

# Initialise SQLite + ChromaDB knowledge base
!python /kaggle/working/Med-I-C/setup_demo.py

## 5 · Launch the App

In [ ]:
%%capture
!pip install -q localtunnel

In [ ]:
import subprocess, threading, time, requests

# Start Streamlit in the background
streamlit_proc = subprocess.Popen(
    ["streamlit", "run", "/kaggle/working/Med-I-C/app.py",
     "--server.port", "8501",
     "--server.headless", "true",
     "--server.enableCORS", "false"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL,
)

# Wait for Streamlit to be ready
for _ in range(15):
    try:
        if requests.get("http://localhost:8501", timeout=2).status_code == 200:
            print("Streamlit is running on port 8501")
            break
    except Exception:
        time.sleep(2)
else:
    print("Streamlit may still be starting…")

In [ ]:
# Expose via localtunnel — the public URL will appear below
tunnel_proc = subprocess.Popen(
    ["npx", "localtunnel", "--port", "8501"],
    stdout=subprocess.PIPE,
    stderr=subprocess.DEVNULL,
    text=True,
)

# Print the public URL
for line in tunnel_proc.stdout:
    if "https://" in line:
        print("\n" + "="*50)
        print(f"  App URL: {line.strip()}")
        print("="*50)
        break